In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from dolfin import *
from osgeo import gdal
import pygmsh
import meshio
import scipy.interpolate as interp
import meshio
from itkwidgets import view
import pyvista as pv
from pyvista import set_plot_theme
set_plot_theme('document')

In [ ]:
"""
# ipython_exit.py
Allows exit() to work if script is invoked with IPython without
raising NameError Exception. Keeps kernel alive.

Use: import variable 'exit' in target script with
     'from ipython_exit import exit'    
"""

import sys
from io import StringIO
from IPython import get_ipython


class IpyExit(SystemExit):
    """Exit Exception for IPython.

    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        # print("exiting")  # optionally print some message to stdout, too
        # ... or do other stuff before exit
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup


def ipy_exit():
    raise IpyExit


if get_ipython():    # ...run with IPython
    exit = ipy_exit  # rebind to custom exit
else:
    exit = exit      # just make exit importable

### Notebook to play with simple 2D groundwater flow with shallow water and Bernouli's approximation

$$ C\frac{\partial h}{\partial t} = \nabla \cdot \left[K(h) \nabla \left( h + z \right)\right] + S $$

* $z$ is the topography of the impermiable base layer.
* $C$ is the drainiable porosity.
* $K(h)$ is conductivity (and is a function of the hydraulic head, but to start with make it constant)

In [ ]:
h_south = 0  # south boundary height
C = 0.34  # drainable porosity
S = 0.03*24/C  # source term mm/hr -> m/day
K = 1e-4*60*60*24/C  # conductivity m/s -> m/day
D = 0  # initial water height m

Build the model mesh based on the Sithas DEM

In [ ]:
# load in DEM
gdal_data = gdal.Open('/work/armitagj/code/caesar-explore/riu-bergantes/paleo-v1/input_data/paleoDEM_v1.tif')
data_array = gdal_data.ReadAsArray().astype(np.float)

# extract contour of the catchment (need to use three segments)
catchment = np.nan_to_num(data_array)
cs = plt.contour(catchment, [0])
plt.show()
xy = cs.collections[0].get_paths()[0]
coor_xy = xy.vertices
xy = cs.collections[0].get_paths()[2]
coor_xy = np.append(coor_xy, xy.vertices, axis=0)
xy = cs.collections[0].get_paths()[1]
coor_xy = np.append(coor_xy, xy.vertices, axis=0)

lx = np.max(coor_xy[:,0])
ly = np.max(coor_xy[:,1])  # characteristic length: ly

# chop off the outflow to make a straight boundary
outflow = np.min(coor_xy[:,1]) + 0.02*ly
coor_xy[np.where(coor_xy[:,1] < outflow), 1] = outflow

coor_z = np.zeros((len(coor_xy), 1))
coor = np.concatenate((coor_xy, coor_z), axis=1)

# create the mesh
geom = pygmsh.built_in.Geometry()
poly = geom.add_polygon(coor[0::100])
i = 0
for line in poly.line_loop.lines:
    geom.add_physical([poly.line_loop.lines[i]], i + 1)
    i = i + 1
geom.add_physical([poly.surface], i + 1)
res = 32
maxcell = ly/res
gmshmesh = pygmsh.generate_mesh(geom, extra_gmsh_arguments=["-2", "-clmax", str(maxcell)])

# export the mesh
cells = np.vstack(np.array([cells.data for cells in gmshmesh.cells
                            if cells.type == "triangle"]))
triangle_mesh = meshio.Mesh(points=gmshmesh.points,
                            cells=[("triangle", cells)])
facet_cells = np.vstack(np.array([cells.data for cells in gmshmesh.cells
                                  if cells.type == "line"]))
facet_data = gmshmesh.cell_data_dict["gmsh:physical"]["line"]
facet_mesh = meshio.Mesh(points=gmshmesh.points,
                         cells=[("line", facet_cells)],
                         cell_data={"name_to_read": [facet_data]})
# export
meshio.xdmf.write("mesh.xdmf", triangle_mesh)
meshio.xdmf.write("facet_mesh.xdmf", facet_mesh)

# read in the mesh for dolfin
mesh = Mesh()
with XDMFFile("mesh.xdmf") as infile:
    infile.read(mesh)
mvc = MeshValueCollection("size_t", mesh, 2)
with XDMFFile("facet_mesh.xdmf") as infile:
    infile.read(mvc, "name_to_read")
mf = cpp.mesh.MeshFunctionSizet(mesh, mvc)

# define function space
V = FunctionSpace(mesh, 'P', 2)
z = Function(V)

# Get the global coordinates
gdim = mesh.geometry().dim()
gc = V.tabulate_dof_coordinates().reshape((-1, gdim))

# Interpolate elevation into the initial condition
ny, nx = np.shape(data_array)
x, y = np.meshgrid(np.linspace(0, nx, nx),
                   np.linspace(0, ny, ny))

flat_data = data_array.flatten()
new_data = flat_data[~np.isnan(flat_data)]
flat_x = x.flatten()[~np.isnan(flat_data)]
flat_y = y.flatten()[~np.isnan(flat_data)]

elevation = interp.griddata((flat_x[0::100], flat_y[0::100]),
                            new_data[0::100],
                            (gc[:, 0], gc[:, 1]),
                            method='nearest')

z.vector()[:] = 1e-1 * elevation  # bedrock elevation from topography
outpoint = np.min(gc[:, 1])
print(outflow)
print(outpoint)
print(np.min(x[1]))

# define boundary
class South(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], outpoint)

In [ ]:
# apply boundary condition (not applied to correct bc for now)
# uS = Constant(h_south)
# South().mark(mf, 1)
# bc = DirichletBC(V, uS, mf, 1)
bc = []

 Check bedrock

In [ ]:
File("mesh.pvd").write(mesh)
test = pv.read("mesh000000.vtu")
p = pv.Plotter(notebook=True, border=False)
p.add_mesh(test, show_edges=True)
p.show()

Try to get stable timestep for this...
$$ \Delta t \leq \frac{\left(\Delta x\right)^2}{2\kappa(h)} $$
the problem is the diffusion coefficient is a function of the hydraulic head. But if I assume the hydraulic head is a maximum of 100m


In [ ]:
hmax = 500  # meters
kappa = K*hmax
cfl = 0.01  # just to be very safe
deltax = mesh.hmin()
dt = cfl * deltax * deltax / (2 * kappa)
print('dt = {} secs; dx = {} m'.format(dt*24*60*60, deltax))

Define the trial $u$ and test $v$ functions and the source term, $S$.

I will assume that I can use the head ($u$) from the previous time-step for the non-linear term within the diffusion: **explicit** I guess.

In [ ]:
def kappa(u):
    return K

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
u_n = interpolate(Constant(D), V)
u_ = u_n
f = Constant(S)

F = u*v*dx + dt * kappa(u_) * dot(grad(u + z), grad(v + z))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)

Solve for the time dependent equation

In [ ]:
# time steps for explicit solutions
u = Function(V)
tol = 1.0e-3        # tolerance
maxiter = 1000        # max no of iterations allowed
tstep = 0
maxtstep = 10

# I will do Piccard iterations to first update the hydraulic head in the kappa(u_) term.
# Once this has reached its tollerance I will update the u_n and move on to the next timestep.

iteration = []
residual = []

while tstep < maxtstep:
    tstep += 1
    print('tstep=%d -------' % (tstep))
    eps = 1.0           # error measure ||u-u_||
    itera = 0           # iteration counter
    while eps > tol and itera < maxiter:
        itera += 1
        solve(a == L, u, bc, solver_parameters={'linear_solver': 'gmres', 'preconditioner': 'ilu'})
        diff = u.vector().get_local() - u_.vector().get_local()
        eps = np.linalg.norm(diff, np.Inf)  # maximum difference (L_inf norm)
        print('iter=%d: norm=%g' % (itera, eps))
        iteration.append(itera)
        residual.append(eps)
        #if itera == maxiter-1:
        #    exit()
        u_.assign(u)
    
    u_n.assign(u)

In [ ]:
plt.figure()
plt.plot(residual, 'k')
plt.show()

In [ ]:
# Save solution to file
file = File("timedep-groundwater-head.pvd")
u.rename("h","head")
file << u
file = File("timedep-groundwater-base.pvd")
z.rename("z","base")
file << z

Plot the solution

In [ ]:
head = pv.read('timedep-groundwater-head000000.vtu')
base = pv.read('timedep-groundwater-base000000.vtu')
head['groundwater'] = base['z'] + head['h']
warp = head.warp_by_scalar(scalars='groundwater',factor=1)
warpz = base.warp_by_scalar(scalars='z',factor=1)
# view(geometries=warp)

In [ ]:
p = pv.Plotter(notebook=False, border=False)
p.add_text("Steady State Solution", font_size=12)
p.add_mesh(warp, scalars='groundwater', cmap='viridis', lighting=True, show_scalar_bar=True, opacity=0.85, color=True)
p.add_mesh(warpz, scalars='z', cmap='magma', lighting=True, show_scalar_bar=True)
p.show_bounds(grid='front', font_size=8, location='outer', all_edges=True)
p.show(use_panel=False,)

In [ ]:
warph = head.warp_by_scalar(scalars='h',factor=10)

In [ ]:
p = pv.Plotter(notebook=False, border=False)
p.add_text("Steady State Solution", font_size=12)
p.add_mesh(warph, scalars='h', cmap='viridis', lighting=True, show_scalar_bar=True)
p.show(use_panel=False,)